In [1]:
!curl -o /tmp/stackex.txt https://raw.githubusercontent.com/Raphtory/Data/main/stackex.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:25 --:--:--     0^C


In [2]:
import pandas as pd
from pyraphtory.context import PyRaphtory
from pyraphtory.input import ImmutableString
from pyraphtory.input import GraphBuilder
from pyraphtory.spouts import FileSpout
from pyraphtory.sources import CSVEdgeListSource
from pyraphtory.sources import Source
from pyraphtory.graph import Row

structure_file = "/tmp/stackex.txt"

ctx = PyRaphtory.local()

In [3]:
graph = ctx.new_graph()

with open(structure_file, 'r') as file :
  for line in file.readlines():
      row = line.split(" ")
      graph.add_edge(int(row[2]),int(row[0]),int(row[1]))




In [4]:
windows = ["1 day", "1 week", "1 month", "1 year"]

df_nodes = graph \
    .walk("1 year") \
    .date_window(windows) \
    .execute(ctx.algorithms.generic.NodeEdgeCount)\
    .to_df(["nodes", "edges_dir", "edges_undir", "temporal_edges"])

In [5]:
df_nodes.tail(20)

,timestamp,window,nodes,edges_dir,edges_undir,temporal_edges
144,1136199338516,1 day,0,0,0,0
145,1136199338516,1 week,0,0,0,0
146,1136199338516,1 month,0,0,0,0
147,1136199338516,1 year,0,0,0,0
148,1167735338516,1 day,0,0,0,0
149,1167735338516,1 week,0,0,0,0
150,1167735338516,1 month,0,0,0,0
151,1167735338516,1 year,0,0,0,0
152,1199271338516,1 day,0,0,0,0
153,1199271338516,1 week,0,0,0,0


In [ ]:
import matplotlib.pyplot as plt
import datetime as dt

fig, ax = plt.subplots()
ax.set_xlabel("Timestamp")
ax.set_ylabel("Number of Temporal Edges")

for window in windows:
    df_sub = df_nodes[df_nodes["window"] == window]
    dates = df_sub["timestamp"].apply(dt.datetime.fromtimestamp)
    ax.plot(dates, df_sub.temporal_edges, label = "window size "+str(window))

ax.legend(loc= "upper left")
plt.show()

In [ ]:
from pyraphtory.scala.implicits.numeric import Long

df_community = graph \
    .execute(ctx.algorithms.generic.community.LPA[Long]())\
    .to_df(["name", "community"])

In [ ]:
df_clustering = graph \
    .walk(86400) \
    .window(windows) \
    .execute(ctx.algorithms.generic.motif.GlobalClusteringCoefficient)\
    .to_df(["avgCluster", "globalCluster"])

In [ ]:
df_clustering.tail()

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt

fig, ax = plt.subplots()
ax.set_xlabel("Timestamp")
ax.set_ylabel("Average Clustering Coefficient")

for window in windows:
    df_sub = df_clustering[df_clustering["window"] == window]
    dates = df_sub["timestamp"].apply(dt.datetime.fromtimestamp)
    ax.plot(dates, df_sub.avgCluster, label = "window size "+str(window))

ax.legend(loc= "upper left")
plt.show()